#Jesús Alberto Aréchiga Carrillo



#Práctica: Visualización de pipelines de scikit-learn



El objetivo de esta práctica es:


1.   Realizar un mini proyecto de clasificación utilizando un pipeline
2.   Visualizar el pipeline
3.   Imputar valores faltantes con la mediana de los datos


## Primero, carga el set de datos "house-prices.csv"

Define los datos y el objetivo. En este caso, recuerda que se debe construir un modelo de clasificación en el que se identifiquen las casas que cuestán más de 125,000 dólares y las cuestan menos o igual a 125,000 dólares.

In [ ]:
import pandas as pd
#Write your code here
ames_housing = pd.read_csv("AmesHousing.csv")
target_name = "SalePrice"
data, target = (
    ames_housing.drop(columns=target_name),
    ames_housing[target_name],
)
target = (target > 125000).astype(int)

Inspecciona las primeras 5 filas del conjunto de datos utilizando `head()`

In [ ]:
ames_housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770.0,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,NaN,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267.0,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160.0,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,NaN,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [ ]:
data['Street'].unique()

array(['Pave', 'Grvl'], dtype=object)

Selecciona las características por inspección considerado las que creas más importantes para la tarea de clasificación del precio de las casas. Puedes probar varias veces hasta seleccionar las que den un mejor accuracy.

In [ ]:
numeric_features = ['Lot Frontage', 'Lot Area', 'Yr Sold', 'Year Built', 'Year Remod/Add', 'Overall Qual', 'Overall Cond']
categorical_features = ['Street', 'Lot Shape', 'Neighborhood', 'Lot Config', 'Bldg Type']
data = data[numeric_features + categorical_features]

Ya seleccionadas las características, muestra los primeros 15 datos y realiza una inspección.

In [ ]:
data.head(15)

,Lot Frontage,Lot Area,Yr Sold,Year Built,Year Remod/Add,Overall Qual,Overall Cond,Street,Lot Shape,Neighborhood,Lot Config,Bldg Type
0,141.0,31770.0,2010,1960,1960,6,5,Pave,IR1,NAmes,Corner,1Fam
1,80.0,NaN,2010,1961,1961,5,6,Pave,Reg,NAmes,Inside,1Fam
2,81.0,14267.0,2010,1958,1958,6,6,Pave,IR1,NAmes,Corner,1Fam
3,93.0,11160.0,2010,1968,1968,7,5,Pave,Reg,NAmes,Corner,1Fam
4,74.0,NaN,2010,1997,1998,5,5,Pave,IR1,Gilbert,Inside,1Fam
5,78.0,9978.0,2010,1998,1998,6,6,Pave,IR1,Gilbert,Inside,1Fam
6,41.0,4920.0,2010,2001,2001,8,5,Pave,Reg,StoneBr,Inside,TwnhsE
7,43.0,5005.0,2010,1992,1992,8,5,Pave,IR1,StoneBr,Inside,TwnhsE
8,39.0,5389.0,2010,1995,1996,8,5,Pave,IR1,StoneBr,Inside,TwnhsE
9,60.0,NaN,2010,1999,1999,7,5,Pave,Reg,Gilbert,Inside,1Fam


¿Observaste datos faltantes (NaN)?. De ser así, muestra cuántos datos faltantes hay y de qué características.

In [ ]:
#Write your code here
data.isna().sum()

,0
Lot Frontage,490
Lot Area,3
Yr Sold,0
Year Built,0
Year Remod/Add,0
Overall Qual,0
Overall Cond,0
Street,0
Lot Shape,0
Neighborhood,0


## Ahora crea el pipeline

Lo primero es definir los pasos del preprocesamiento: aplica un escalamiento estándar y un imputador de datos (en caso de ser necesario) para las características numéricas. Así como una codificación one-hot para las características categóricas.

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        (
            "scaler",
            StandardScaler(),
        ),
    ]
)

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

El siguiente paso es aplicar las transformaciones usando `ColumnTransformer`

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

Ahora define el modelo con un pipeline que incluya el preprocesamiento y el clasificador de regresión logística.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression()),
    ]
)

Muestra la visualización del modelo

In [ ]:
#Write your code here
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Lot Frontage', 'Lot Area',
                                                   'Yr Sold', 'Year Built',
                                                   'Year Remod/Add',
                                                   'Overall Qual',
                                                   'Overall Cond']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Street', 'Lot Shape',
                                                   'Neighborhood', 'Lot Config',
                                                   'Bldg Type'])])),
                ('classifier', LogisticRegression())])

## Finalmente, entrena el modelo y valida el desempeño utilizando validación cruzada

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
scores = cv_results["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.888 ± 0.010


Para comprobar si tu procedimiento fue correcto, asegúrate de obtener un accuracy igual o mayor a 0.87. Si tu accuracy fue menor, prueba con otra selección de características y vuelve a entrenar el modelo.



Escribe tus conclusiones sobre esta práctica y sobre los resultados del desempeño del modelo.

Se pueden llenar los datos faltantes (NaN) utilizando imputación con la media o la mediana de datos categóricos.
El resultado del entrenamiento varía dependiendo de qué datos numéricos y categóricos se utilizan.